In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%writefile .env

TAVILY_API_KEY = your_key
JINA_API_KEY = your_key
CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
BROWSER_HEADLESS=False


Overwriting .env


In [3]:
import sys
from src.workflow import run_agent_workflow

In [4]:
user_query = '''
    I would like to analyze Amazon product sales data.
    The target for analysis is the './data/Amazon_Sale_Report.csv' file.
    Please conduct an analysis to extract marketing insights based on this data.
    Please perform various analytical techniques starting from basic data attribute exploration, product sales trends, variable relationships, variable combinations, etc.
    If there are any additional analyses needed to extract insights after the data analysis, please perform those as well.
    Please include detailed analysis in the report along with supporting images and charts.
    Please save the final report in PDF format.
'''

user_query = '''
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.
'''

user_query = '''
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.
'''

#user_query = "안녕 나는 장동진이야"
#user_query = '''
#    이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 간단하게 1개정도만 해 주세요. 코드 테스트 중이거든요 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.
#    '''

# user_query = '''
#     영문으로 작성된 pptx 파일을 한글로 변환하고 싶습니다.
#     대상은 "./data/bedrock.pptx" 파일 입니다.
#     슬라이드의 영문을 한글로 번역하고 슬라이드 structure는 유지해 주세요.
# '''

# user_query = '''
#     주어진 프로젝트 소스코드는 "Bedrock-Manus: AI automation framework optimized for Amazon Bedrock and business use cases 입니다."
#     프로젝트 소스코드는 "./data/08_bedrock_manus.tar.gz" 파일 입니다.
#     나는 Streamlit을 이용해서 이 소스코드에 대한 UI를 만들고 싶습니다. 
#     제가 원하는 요구사항은
#     1. 현재 스텝이 무엇인지 (coodinator, supervisor, planner 등등) 보이는 곳이 있으면 좋겠습니다.
#         - 스텝에 대한 자세한 정보는 "./README.md"를 참고하시면 됩니다. 
#     2. 각 스텝에서 진행되는 것이 보였으면 좋겠습니다. 즉, 각 스텝에서 llm이 생성하는 토큰들을 실시간으로 볼 수 있으면 좋겠습니다. 
#     3. "app.py" 파일만 생성하지 말고, 필요하다면 기존의 다른 파일 (workflow.py 등)들도  수정하세요. 
#     4. 수정된 파일은 모두 ./artifacts 에 넣어주세요.
#     참고할 사항이 있습니다. 
#     1. 소스코드 수행은 "main.py"로 수행됩니다. 
#     2. "./src/service/workflow_service.py" 이 파일은 참고하지 마세요. 

#     UI 생성을 위한 프로젝트 코드를 작성해주세요
#     결과물은 ./artifacts 에 넣어주세요.
# '''

# user_query = '''
#     "2025년 중국 가전 신제품 출시에 대한 리포트 작성해줘."
# '''

In [9]:
import os
import shutil

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")


import nest_asyncio
nest_asyncio.apply()

remove_artifact_folder()

# result = run_agent_workflow(
#     user_input=user_query,
#     debug=False
# )
import asyncio
result = asyncio.run(run_agent_workflow(
    user_input=user_query,
    debug=False
))


#슈퍼바이저 에서 코더, 트랙커, 리포터 툴로 만들기 
#코더부터 툴(에이전트 에즈 어 툴)로 만들고 
#트랙커는 새로 만들기


# Print the conversation history
print("\n=== Conversation History ===")
print ("result", result)
# for history in result["history"]:

#     print ("===")
#     print (f'agent: {history["agent"]}')
#     print (f'message: {history["message"]}')


INFO [src.workflow] ===== Starting workflow =====

INFO [src.workflow] 
user input: 
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.


INFO [src.workflow] ===== Workflow completed successfully =====

INFO [src.graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled


'./artifacts/' 폴더가 존재하지 않습니다.
handoff_to_planner: I'll need to consult our planning system for this request.

이 요청은 데이터 분석, 시각화, PDF 리포트 생성 등 복잡한 작업이 포함되어 있어 플래너 시스템의 도움이 필요합니다. 플래너가 체계적인 분석 계획을 수립하고 실행할 수 있도록 하겠습니다.


INFO [src.graph.nodes] ===== Coordinator updated shared global state with 5 keys =====

INFO [src.graph.nodes] ===== Coordinator completed task =====

INFO [src.graph.nodes] ===== Planner generating plan =====

INFO [src.graph.nodes] 
Planner received task:
[ {'text': 'Original Task:'},
  'request',
  'request_prompt',
  {'text': '\nInputs from previous nodes:'},
  {'text': '\nFrom coordinator:'},
  { 'text': "  - Agent: handoff_to_planner: I'll need to consult our planning system for this "
            'request.\n'
            '\n'
            '이 요청은 데이터 분석, 시각화, PDF 리포트 생성 등 복잡한 작업이 포함되어 있어 플래너 시스템의 도움이 필요합니다. 플래너가 체계적인 분석 계획을 '
            '수립하고 실행할 수 있도록 하겠습니다.\n'}]

INFO [src.graph.nodes] 
Planner received kwargs:
{}

INFO [src.graph.nodes] ===== Successfully retrieved shared state from global storage =====

INFO [src.graph.nodes] 
Shared state:
{ 'goto': 'planner',
  'history': [ { 'agent': 'coordinator',
                 'message': "handoff_to_planner: I'll need to consult our 

사용자의 요청을 분석해보겠습니다:

1. 아마존 상품판매 데이터 분석
2. "./data/Dat-fresh-food-claude.csv" 파일 대상
3. 마케팅 인사이트 추출을 위한 분석 수행
4. 분석 범위: 기본적인 데이터 속성 탐색, 상품 판매 트렌드, 변수 관계, 변수 조합
5. 필요시 추가 분석 수행
6. 상세 분석과 시각화(차트, 이미지)를 포함한 리포트 작성
7. 최종 리포트는 PDF 형태로 저장

이 작업을 효율적으로 수행하기 위해 다음과 같은 계획을 세워보겠습니다:

1. Coder를 통해 데이터 로드, 전처리, 탐색적 분석, 시각화 등 모든 데이터 분석 작업을 수행합니다.
2. 분석이 완료된 후 Reporter를 통해 최종 보고서를 PDF 형태로 작성합니다.

각 단계를 더 자세히 계획해 보겠습니다:

Coder의 작업:
1. 데이터 로드 및 기본 정보 확인
   - CSV 파일 로드
   - 데이터 크기, 컬럼 확인
   - 기본 통계 확인 (요약 통계량)
   - 결측치, 중복값 등 데이터 품질 확인

2. 기초 탐색적 분석
   - 데이터 유형 및 분포 확인
   - 히스토그램, 박스 플롯 등을 통한 변수별 분포 시각화
   - 상관관계 분석

3. 심화 데이터 분석
   - 상품 판매 트렌드 분석
   - 변수 간 관계 분석
   - 주요 변수 조합에 따른 분석
   - 시계열 패턴 분석 (판매 데이터의 경우)

4. 마케팅 인사이트 도출을 위한 추가 분석
   - 고객 세그먼트 분석 (가능한 경우)
   - 판매 패턴 분석
   - 상품 카테고리별 분석
   - 주요 성과 지표 분석

5. 시각화 작업
   - 주요 인사이트를 시각적으로 표현할 차트 및 그래프 작성
   - 결과 해석을 위한 주석 추가

Reporter의 작업:
1. 분석 결과를 바탕으로 종합 보고서 작성
2. 주요 마케팅 인사이트 요약
3. 데이터 분석 방법론 및 결과 설명
4. 시각화 자료 포함
5. PDF 형태로 보고서 저장

이제 이 계획을 체크리스트 형태로 


INFO [src.graph.nodes] ===== Planner analyzing and creating execution plan =====

INFO [src.graph.nodes] ===== Planner completed plan generation =====

INFO [src.graph.nodes] ===== Supervisor evaluating next action =====

INFO [src.graph.nodes] 
Supervisor received task:
[ {'text': 'Original Task:'},
  'request',
  'request_prompt',
  {'text': '\nInputs from previous nodes:'},
  {'text': '\nFrom planner:'},
  { 'text': '  - Agent: # Plan\n'
            '\n'
            '## thought\n'
            '이 프로젝트는 아마존 상품판매 데이터를 분석하여 마케팅 인사이트를 도출하는 작업입니다. 분석은 '
            '"./data/Dat-fresh-food-claude.csv" 파일을 대상으로 합니다. 기본적인 데이터 탐색부터 심화 분석까지 수행한 후, 최종적으로 '
            'PDF 형태의 보고서로 작성해야 합니다. 작업을 Coder와 Reporter에게 효율적으로 분배하여 진행하겠습니다.\n'
            '\n'
            '## title:\n'
            '아마존 신선식품 판매 데이터 분석 및 마케팅 인사이트 도출\n'
            '\n'
            '## steps:\n'
            '### 1. Coder: 아마존 판매 데이터 종합 분석\n'
            '- [ ] "./data/Dat-fresh-food-claude.csv" 파일 로드 및 데이터 기본 정보 확인 (크기, 

...


INFO [bash_tool] ===== Executing Bash =====

INFO [bash_tool] ===== Coder - Command: ls -la . =====

INFO [src.tools.decorators] 
 Tool handle_bash_tool returned:
ls -la .||total 188
drwxrwxr-x  9 ubuntu ubuntu  4096 Aug 23 11:31 .
drwxrwxr-x  8 ubuntu ubuntu  4096 Aug 21 04:12 ..
-rw-rw-r--  1 ubuntu ubuntu   158 Aug 23 11:26 .env
-rw-rw-r--  1 ubuntu ubuntu  6041 Aug 19 01:34 CLAUDE.md
-rw-rw-r--  1 ubuntu ubuntu  2603 Aug 15 11:43 CONTRIBUTING.md
-rw-rw-r--  1 ubuntu ubuntu  1065 Aug 15 11:43 LICENSE
-rw-rw-r--  1 ubuntu ubuntu  7018 Aug 15 11:43 README.md
drwxrwxr-x  2 ubuntu ubuntu  4096 Aug 15 11:43 app
drwxrwxr-x  2 ubuntu ubuntu  4096 Aug 23 11:31 artifacts
drwxrwxr-x  2 ubuntu ubuntu  4096 Aug 15 11:43 assets
drwxrwxr-x  2 ubuntu ubuntu  4096 Aug 23 11:30 data
drwxrwxr-x  2 ubuntu ubuntu  4096 Aug 15 11:43 docs
-rw-rw-r--  1 ubuntu ubuntu 12790 Aug 19 07:00 final_report.md
-rw-rw-r--  1 ubuntu ubuntu 77800 Aug 23 11:29 main.ipynb
-rw-rw-r--  1 ubuntu ubuntu  1737 Aug 15 21:45

죄송합니다만, 지정된 데이터 파일(Dat-fresh-food-claude.csv)이 현재 디렉토리 구조에서 찾을 수 없습니다. 분석을 진행하기 위해서는 다음 중 하나의 조치가 필요합니다:

1. 데이터 파일을 ./data/ 디렉토리에 업로드해주시거나
2. 데이터 파일의 정확한 경로를 알려주시기 바랍니다.

데이터 파일이 준비되면 계획에 따라 바로 분석을 시작하도록 하겠습니다. 어떻게 진행하면 좋을지 안내해 주시겠습니까?


INFO [coder_agent_tool] ===== Updated shared state with coder results =====

INFO [coder_agent_tool] ===== Coder Agent Tool completed successfully =====


result_text 데이터 파일의 존재 여부를 확인하고 필요한 디렉토리를 생성해보겠습니다.현재 ./data/ 디렉토리에 필요한 CSV 파일이 없는 것을 확인했습니다. 상위 디렉토리도 확인해보겠습니다.죄송합니다만, 지정된 데이터 파일(Dat-fresh-food-claude.csv)이 현재 디렉토리 구조에서 찾을 수 없습니다. 분석을 진행하기 위해서는 다음 중 하나의 조치가 필요합니다:

1. 데이터 파일을 ./data/ 디렉토리에 업로드해주시거나
2. 데이터 파일의 정확한 경로를 알려주시기 바랍니다.

데이터 파일이 준비되면 계획에 따라 바로 분석을 시작하도록 하겠습니다. 어떻게 진행하면 좋을지 안내해 주시겠습니까?
데이터 생성 요청이 계속 실패하고 있습니다. 더 간단하게 접근해보겠습니다.

KeyboardInterrupt: 